In [1]:
%matplotlib inline
import numpy as np
!pip install dill
!pip install surmise
import os
from os import path
import sys
import dill
import pickle
sys.path.insert(0, path.abspath('./'))

from src.emulator_BAND import EmulatorBAND
#from src.emulator import Emulator


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Define function to create pickle file with training dataset and validation test points

Use the last 5 parameters as validation. Later we might add posterior points.

In [7]:
def read_config(file):
    with open(file, 'r') as f:
        # Initialize an empty dictionary to store the values
        values = []
        # Read the file line by line
        for line in f:
            # If the line starts with a '#', it's a comment
            if line.startswith('#'):
                # Split the line at the colon
                parts = line.split(':')
                if len(parts) < 2:
                    continue
                # The key is the part before the colon, stripped of leading/trailing whitespace and the '#'
                key = parts[0].strip().lstrip('#')
                # The value is the part after the colon, stripped of leading/trailing whitespace
                value = float(parts[1].strip())
                # Add the key-value pair to the dictionary
                values.append(value)
    return values


import pandas as pd
import numpy as np

####################
#GLOBAL VARIABLES
#num_points=500
num_points=4
####################



#generate the data pickles
path_data = '../training_points/raw'
path_configs = '../training_points/configs'
path_output = '../latent_pickled/'
datasets = [
    'exp_19.6_05_eta_spectra', 'exp_200_05_y_spectra_piminus',
    'exp_19.6_05_integrated', 'exp_200_05_y_spectra_piplus',
    'exp_19.6_05_pT_spectra_kminus', 'exp_200_1525_eta_spectra',
    'exp_19.6_05_pT_spectra_kplus', 'exp_200_1525_phobos_v2_spectra',
    'exp_19.6_05_pT_spectra_p', 'exp_200_2030_integrated',
    'exp_19.6_05_pT_spectra_pbar', 'exp_200_2030_phenix_pT_v2_spectra',
    'exp_19.6_05_pT_spectra_piminus', 'exp_200_2030_phenix_pT_v3_spectra',
    'exp_19.6_05_pT_spectra_piplus', 'exp_200_2030_pT_spectra_kminus',
    'exp_19.6_1525_eta_spectra', 'exp_200_2030_pT_spectra_kplus','exp_200_2030_pT_spectra_p',
    'exp_19.6_2030_integrated', 'exp_200_2030_pT_spectra_pbar',
    'exp_19.6_2030_pT_spectra_kminus', 'exp_200_2030_pT_spectra_piminus',
    'exp_19.6_2030_pT_spectra_kplus', 'exp_200_2030_pT_spectra_piplus',
    'exp_19.6_2030_pT_spectra_p', 'exp_7.7_05_integrated',
    'exp_19.6_2030_pT_spectra_pbar', 'exp_7.7_05_pT_spectra_kminus',
    'exp_19.6_2030_pT_spectra_piminus', 'exp_7.7_05_pT_spectra_kplus',
    'exp_19.6_2030_pT_spectra_piplus', 'exp_7.7_05_pT_spectra_p',
    'exp_19.6_2030_star_v2_pT_spectra', 'exp_7.7_05_pT_spectra_pbar',
    'exp_200_05_eta_spectra', 'exp_7.7_05_pT_spectra_piminus',
    'exp_200_05_integrated', 'exp_7.7_05_pT_spectra_piplus',
    'exp_200_05_pT_spectra_kminus', 'exp_7.7_2030_integrated',
    'exp_200_05_pT_spectra_kplus', 'exp_7.7_2030_pT_spectra_kminus',
    'exp_200_05_pT_spectra_p', 'exp_7.7_2030_pT_spectra_kplus',
    'exp_200_05_pT_spectra_pbar', 'exp_7.7_2030_pT_spectra_p',
    'exp_200_05_pT_spectra_piminus', 'exp_7.7_2030_pT_spectra_pbar',
    'exp_200_05_pT_spectra_piplus', 'exp_7.7_2030_pT_spectra_piminus',
    'exp_200_05_y_spectra_kminus', 'exp_7.7_2030_pT_spectra_piplus',
    'exp_200_05_y_spectra_kplus', 'exp_7.7_2030_star_v2_pT_spectra'
]
list_of_pickles=[]
integrated_values=[
    "dNdy_kminus",
    "dNdy_kplus",
    "dNdy_p",
    "dNdy_pbar",
    "dNdy_piminus",
    "dNdy_piplus",
    "meanpT_kminus",
    "meanpT_kplus",
    "meanpT_p",
    "meanpT_pbar",
    "meanpT_piminus",
    "meanpT_piplus",
    "star_v2",
    "star_v3"
]
print("RAndom variation added!")
import random 
for dataset in datasets:
    if "integrated" not in dataset:
        dict_df_full={}
        for i in range(num_points):
            current_path = path_data +"/"+str(i)+"/"+dataset
            config_path = path_configs +"/config_sets_run_"+str(i+1).zfill(3)+".yaml"
            df = pd.read_csv(current_path)
            dict_df={"obs":np.asarray([df.iloc[:, -2]*np.random.uniform(0.8, 2, df.shape[0])+np.random.uniform(0., 1e-10, df.shape[0]),df.iloc[:, -1]]), "parameter": read_config(config_path)}
            #print(read_config(config_path))
            dict_df_full[i]=dict_df
        with open( path_output+dataset+".pkl", 'wb') as f:
            pickle.dump(dict_df_full, f)
        list_of_pickles.append(path_output+dataset+".pkl")
    elif "integrated" in dataset:
        for key in integrated_values:
            dict_df_full={}
            for i in range(num_points):
                current_path = path_data +"/"+str(i)+"/"+dataset
                config_path = path_configs +"/config_sets_run_"+str(i+1).zfill(3)+".yaml"
                df = pd.read_csv(current_path)
                dict_df={"obs":np.asarray([df[key].values*random.uniform(0.8, 2),df[key+"_error"].values]), "parameter": read_config(config_path)}
                dict_df_full[i]=dict_df
            with open(path_output + dataset + "_" + key + ".pkl", 'wb') as f:
                pickle.dump(dict_df_full, f)
            list_of_pickles.append(path_output+dataset+"_"+key+".pkl")


print(list_of_pickles)


RAndom variation added!
['../latent_pickled/exp_19.6_05_eta_spectra.pkl', '../latent_pickled/exp_200_05_y_spectra_piminus.pkl', '../latent_pickled/exp_19.6_05_integrated_dNdy_kminus.pkl', '../latent_pickled/exp_19.6_05_integrated_dNdy_kplus.pkl', '../latent_pickled/exp_19.6_05_integrated_dNdy_p.pkl', '../latent_pickled/exp_19.6_05_integrated_dNdy_pbar.pkl', '../latent_pickled/exp_19.6_05_integrated_dNdy_piminus.pkl', '../latent_pickled/exp_19.6_05_integrated_dNdy_piplus.pkl', '../latent_pickled/exp_19.6_05_integrated_meanpT_kminus.pkl', '../latent_pickled/exp_19.6_05_integrated_meanpT_kplus.pkl', '../latent_pickled/exp_19.6_05_integrated_meanpT_p.pkl', '../latent_pickled/exp_19.6_05_integrated_meanpT_pbar.pkl', '../latent_pickled/exp_19.6_05_integrated_meanpT_piminus.pkl', '../latent_pickled/exp_19.6_05_integrated_meanpT_piplus.pkl', '../latent_pickled/exp_19.6_05_integrated_star_v2.pkl', '../latent_pickled/exp_19.6_05_integrated_star_v3.pkl', '../latent_pickled/exp_200_05_y_spectra_pi

In [3]:
#do they have to be pickels?
#the pickels contain dictonaries. the dictionary is [{obs: [[val1, val2...], [err1,err2...]], parameter: i}] with length design points
"""
dataDict = {
    'event_id1': { #id of parameter points
        'obs': [[...], [...]],  # 2D array-like structure, values and errors
        'parameter': ...  # parameter list, just pure numerical values
    },
    'event_id2': {
        'obs': [[...], [...]],  # 2D array-like structure
        'parameter': ...  # Scalar or 1D array
    },
    ...
}
"""
#print("The following pickles have been created:")
datasets = list_of_pickles
path_output_train = '../latent_train/'
path_output_val = '../latent_val/'
# Check if the output folder exists, if not, create it
if not os.path.exists(path_output_train):
    os.makedirs(path_output_train)

if not os.path.exists(path_output_val ):
    os.makedirs(path_output_val )

def check_file_length(filename, expected_length):
    with open(f"{path_output}{filename}", 'rb') as f:
        data = pickle.load(f)
    if len(data) == expected_length:
        pass
    else:
        print(f"{filename} does not have the correct length. Expected: {expected_length}, Actual: {len(data)}")

datasets_train = []
for dataset in datasets:
    current_path = dataset
    #print(dataset)
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)
    
    #print(f"The dataset {dataset} has length {len(data)}.")
    # Separate data based on event ID
    sorted_event_ids = sorted(data.keys(), key=lambda x: int(x))
    first_data = {event_id: data[event_id] for event_id in sorted_event_ids[:num_points-1]}
    second_data = {event_id: data[event_id] for event_id in sorted_event_ids[num_points-1:]}
    #print(f'{dataset.split(".p")[0].split("/")[-1]}_train.pkl')
    # Save separated data to pickle files
    with open(f'{path_output_train}{dataset.split(".p")[0].split("/")[-1]}_train.pkl', 'wb') as pf1:
        pickle.dump(first_data, pf1)
        
    with open(f'{path_output_val}{dataset.split(".p")[0].split("/")[-1]}_val.pkl', 'wb') as pf2:
        pickle.dump(second_data, pf2)

    check_file_length(f'{path_output_train}{dataset.split(".p")[0].split("/")[-1]}_train.pkl', num_points-1)
    datasets_train.append(f'{path_output_train}{dataset.split(".p")[0].split("/")[-1]}_train.pkl')
    check_file_length(f'{path_output_val}{dataset.split(".p")[0].split("/")[-1]}_val.pkl', 1) 

# Perform the training of the emulators for all datasets

After training the emulators, we save them with `dill`, such that they can be reloaded from file for the MCMC later.

Perform the training without the `parameterTrafoPCA` (for $\zeta/s(T)$, $\eta/s(\mu_B)$, $\langle y_{\rm loss}\rangle(y_{\rm init})$).

In [32]:
#model_par contains the full range of values for the parameters
model_par = '../training_points/configs/config_AuAu_200_bulk_scan_central.yaml'
path_input = '../latent_train/'
path_output = '../trained_emulators_no_PCA/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

for dataset in datasets_train:
    emu1 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCGP', logTrafo=False, parameterTrafoPCA=False)

    emu1.trainEmulatorAutoMask()
    emu2 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCSK', logTrafo=False, parameterTrafoPCA=False)
    emu2.trainEmulatorAutoMask()


    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCGP_trained.sav', 'wb') as f:
        dill.dump(emu1, f)
    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCSK_trained.sav', 'wb') as f:
        dill.dump(emu2, f)

[INFO][emulator_BAND] loading training data from ../latent_train/../latent_train/exp_19.6_05_eta_spectra_train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[1.0569, 0.1302500000000002, 0.6737500000000001, 0.2625, 0.146871, 1.7194, 0.16005, 0.762, 1.334, 0.15436, 0.054200000000000005, 15.7865, 0.036065, 0.10282, 1.1692]
[1.1535, -0.1997500000000001, -1.13075, 0.3315, 0.12332900000000001, 2.7910000000000004, 0.23204999999999995, 1.1300000000000001, 0.354, 0.16732, 0.0606, 39.2495, 0.06988500000000002, 0.11961999999999999, 0.93]
[1.3256999999999999, 1.4722499999999998, 0.48475, 0.5755, 0.074755, 0.4958, 0.28425, 0.534, 2.0300000000000002, 0.2494, 0.18860000000000002, 9.1505, 0.08356500000000001, 0.09133999999999998, 1.0108]
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 3, discarded points: 0
[INFO][emulator_BAND] Performing emulator training ...
[INFO][emulator_BAND] Train 

Perform the training with the `parameterTrafoPCA` (for $\zeta/s(T)$, $\eta/s(\mu_B)$, $\langle y_{\rm loss}\rangle(y_{\rm init})$).

In [4]:
model_par = '../training_points/configs/config_AuAu_200_bulk_scan_central.yaml'
path_input = '../latent_train/'
path_output = '../trained_emulators_PCA/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

for dataset in datasets_train:
    emu1 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCGP', logTrafo=False, parameterTrafoPCA=True)
    emu1.trainEmulatorAutoMask()
    emu2 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCSK', logTrafo=False, parameterTrafoPCA=True)
    emu2.trainEmulatorAutoMask()
    

    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCGP_trained.sav', 'wb') as f:
        dill.dump(emu1, f)
    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCSK_trained.sav', 'wb') as f:
        dill.dump(emu2, f)
    

#TODO: HOW MANY PCA COMPONENTS?

[INFO][emulator_BAND] loading training data from ../latent_train/../latent_train/exp_19.6_05_eta_spectra_train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[1.0569, 0.1302500000000002, 0.6737500000000001, 0.2625, 0.146871, 1.7194, 0.16005, 0.762, 1.334, 0.15436, 0.054200000000000005, 15.7865, 0.036065, 0.10282, 1.1692]
[1.1535, -0.1997500000000001, -1.13075, 0.3315, 0.12332900000000001, 2.7910000000000004, 0.23204999999999995, 1.1300000000000001, 0.354, 0.16732, 0.0606, 39.2495, 0.06988500000000002, 0.11961999999999999, 0.93]
[1.3256999999999999, 1.4722499999999998, 0.48475, 0.5755, 0.074755, 0.4958, 0.28425, 0.534, 2.0300000000000002, 0.2494, 0.18860000000000002, 9.1505, 0.08356500000000001, 0.09133999999999998, 1.0108]
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 3, discarded points: 0
[INFO][emulator_BAND] Prepare bulk viscosity parameter PCA ...
[INFO][emulator_BAND

In [5]:
model_par = '../training_points/configs/config_AuAu_200_bulk_scan_central.yaml'
path_input = '../latent_train/'
path_output = '../trained_emulators_LOG/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

for dataset in datasets_train:
    emu1 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCGP', logTrafo=True, parameterTrafoPCA=False)
    emu1.trainEmulatorAutoMask()
    emu2 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCSK', logTrafo=True, parameterTrafoPCA=False)
    emu2.trainEmulatorAutoMask()
    

    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCGP_trained.sav', 'wb') as f:
        dill.dump(emu1, f)
    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCSK_trained.sav', 'wb') as f:
        dill.dump(emu2, f)
    


[INFO][emulator_BAND] loading training data from ../latent_train/../latent_train/exp_19.6_05_eta_spectra_train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[1.0569, 0.1302500000000002, 0.6737500000000001, 0.2625, 0.146871, 1.7194, 0.16005, 0.762, 1.334, 0.15436, 0.054200000000000005, 15.7865, 0.036065, 0.10282, 1.1692]
[1.1535, -0.1997500000000001, -1.13075, 0.3315, 0.12332900000000001, 2.7910000000000004, 0.23204999999999995, 1.1300000000000001, 0.354, 0.16732, 0.0606, 39.2495, 0.06988500000000002, 0.11961999999999999, 0.93]
[1.3256999999999999, 1.4722499999999998, 0.48475, 0.5755, 0.074755, 0.4958, 0.28425, 0.534, 2.0300000000000002, 0.2494, 0.18860000000000002, 9.1505, 0.08356500000000001, 0.09133999999999998, 1.0108]
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 3, discarded points: 0
[INFO][emulator_BAND] Performing emulator training ...
[INFO][emulator_BAND] Train 

In [6]:
model_par = '../training_points/configs/config_AuAu_200_bulk_scan_central.yaml'
path_input = '../latent_train/'
path_output = '../trained_emulators_LOG_PCA/'


# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

for dataset in datasets_train:
    emu1 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCGP', logTrafo=True, parameterTrafoPCA=True)
    emu1.trainEmulatorAutoMask()
    emu2 = EmulatorBAND(f'{path_input}{dataset}', model_par, method='PCSK', logTrafo=True, parameterTrafoPCA=True)
    emu2.trainEmulatorAutoMask()
    

    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCGP_trained.sav', 'wb') as f:
        dill.dump(emu1, f)
    with open(f'{path_output}{dataset.split(".p")[0].split("/")[-1]}_PCSK_trained.sav', 'wb') as f:
        dill.dump(emu2, f)

[INFO][emulator_BAND] loading training data from ../latent_train/../latent_train/exp_19.6_05_eta_spectra_train.pkl ...
[INFO][emulator_BAND] Not discarding parameters with high errors. To enable this, we have to add cuts!
[1.0569, 0.1302500000000002, 0.6737500000000001, 0.2625, 0.146871, 1.7194, 0.16005, 0.762, 1.334, 0.15436, 0.054200000000000005, 15.7865, 0.036065, 0.10282, 1.1692]
[1.1535, -0.1997500000000001, -1.13075, 0.3315, 0.12332900000000001, 2.7910000000000004, 0.23204999999999995, 1.1300000000000001, 0.354, 0.16732, 0.0606, 39.2495, 0.06988500000000002, 0.11961999999999999, 0.93]
[1.3256999999999999, 1.4722499999999998, 0.48475, 0.5755, 0.074755, 0.4958, 0.28425, 0.534, 2.0300000000000002, 0.2494, 0.18860000000000002, 9.1505, 0.08356500000000001, 0.09133999999999998, 1.0108]
[INFO][emulator_BAND] All training data are loaded.
[INFO][emulator_BAND] Training dataset size: 3, discarded points: 0
[INFO][emulator_BAND] Prepare bulk viscosity parameter PCA ...
[INFO][emulator_BAND

Generate an 'experimental' dataset from one of the posterior points for closure testing.

In [3]:
# path_data = './separate_training_posterior_data_1095/'
# path_output = './separate_training_posterior_data_1095/'
# datasets_posterior = [
#             'AuAu200_dNdy_posterior.pkl',
#             'AuAu200_PHOBOSdNdeta_posterior.pkl',
#             'AuAu200_PHOBOSv2eta_posterior.pkl',
#             'AuAu200_pTvn_posterior.pkl',
#             'AuAu19p6_dNdy_posterior.pkl',
#             'AuAu19p6_PHOBOSdNdeta_posterior.pkl',
#             'AuAu19p6_pTvn_posterior.pkl',
#             'AuAu7.7_dNdy_posterior.pkl',
#             'AuAu7.7_pTvn_posterior.pkl']

# # Check if the output folder exists, if not, create it
# if not os.path.exists(path_output):
#     os.makedirs(path_output)

# event_data = []
# for dataset in datasets_posterior:
#     current_path = path_data + dataset
#     with open(current_path, "rb") as pf:
#         data = pickle.load(pf)

#     # Get the first event from the posterior dataset
#     test_data = {event_id: data[event_id] for event_id in sorted(data.keys())[4:5]}
#     event_data.append(test_data)


# for event_dict in event_data[1:]:
#     # Get the 'obs' array for the current event
#     obs_array_new = event_dict['1099']['obs']
    
#     # Extend the 'obs' array of the first element with the values from the current event
#     event_data[0]['1099']['obs'] = np.concatenate((event_data[0]['1099']['obs'], obs_array_new), axis=1)

# # Save separated data to pickle files
# with open(f'{path_output}example_data_test_point1099.pkl', 'wb') as pf1:
#     pickle.dump(event_data[0], pf1)


In [2]:
# path_data = './separate_training_posterior_data_1095/'
# path_output = './separate_training_posterior_data_1095/'
# datasets_posterior = [
#             'AuAu200_dNdy_posterior.pkl',
#             'AuAu200_PHOBOSdNdeta_posterior.pkl',
#             'AuAu200_PHOBOSv2eta_posterior.pkl',
#             'AuAu200_pTvn_posterior.pkl',
#             'AuAu19p6_dNdy_posterior.pkl',
#             'AuAu19p6_PHOBOSdNdeta_posterior.pkl',
#             'AuAu19p6_pTvn_posterior.pkl',
#             'AuAu7.7_dNdy_posterior.pkl',
#             'AuAu7.7_pTvn_posterior.pkl']

# # Check if the output folder exists, if not, create it
# if not os.path.exists(path_output):
#     os.makedirs(path_output)

# event_data = []
# for dataset in datasets_posterior:
#     current_path = path_data + dataset
#     with open(current_path, "rb") as pf:
#         data = pickle.load(pf)

#     # Get the first event from the posterior dataset
#     test_data = {event_id: data[event_id] for event_id in sorted(data.keys())[2:3]}
#     event_data.append(test_data)


# for event_dict in event_data[1:]:
#     # Get the 'obs' array for the current event
#     obs_array_new = event_dict['1097']['obs']
    
#     # Extend the 'obs' array of the first element with the values from the current event
#     event_data[0]['1097']['obs'] = np.concatenate((event_data[0]['1097']['obs'], obs_array_new), axis=1)

# # Save separated data to pickle files
# with open(f'{path_output}example_data_test_point1097.pkl', 'wb') as pf1:
#     pickle.dump(event_data[0], pf1)


Generate the point for the test of the logarithmic training

In [5]:
path_data = './separate_training_posterior_data_1095/'
path_output = './separate_training_posterior_data_1095/'
datasets_posterior = [
            'AuAu200_dNdy_posterior.pkl',
            'AuAu200_PHOBOSdNdeta_posterior.pkl',
            'AuAu200_PHOBOSv2eta_posterior.pkl',
            'AuAu200_pTvn_posterior.pkl',
            'AuAu19p6_dNdy_posterior.pkl',
            'AuAu19p6_PHOBOSdNdeta_posterior.pkl',
            'AuAu19p6_pTvn_posterior.pkl',
            'AuAu7.7_dNdy_posterior.pkl',
            'AuAu7.7_pTvn_posterior.pkl']

# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

event_data = []
for dataset in datasets_posterior:
    current_path = path_data + dataset
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)

    # Get the first event from the posterior dataset
    test_data = {event_id: data[event_id] for event_id in sorted(data.keys())[4:5]}
    event_data.append(test_data)

# modify the datasets with the multiplicities and take the log
datasets_to_modify = [0,1,4,5,7]
for i in datasets_to_modify:
    event_data[i]['1099']['obs'][0,:] = np.log(np.abs(event_data[i]['1099']['obs'][0,:]) + 1e-30)
    event_data[i]['1099']['obs'][1,:] = np.abs(event_data[i]['1099']['obs'][1,:]/event_data[i]['1099']['obs'][0,:] + 1e-30)

for event_dict in event_data[1:]:
    # Get the 'obs' array for the current event
    obs_array_new = event_dict['1099']['obs']
    
    # Extend the 'obs' array of the first element with the values from the current event
    event_data[0]['1099']['obs'] = np.concatenate((event_data[0]['1099']['obs'], obs_array_new), axis=1)

# Save separated data to pickle files
with open(f'{path_output}example_data_test_point1099_LOG.pkl', 'wb') as pf1:
    pickle.dump(event_data[0], pf1)

Generate one dataset from all of the training and posterior points.

In [8]:
path_data = '../data/'
path_output = './full_data_one_pkl/'
datasets_posterior = [
            'AuAu200_dNdy.pkl',
            'AuAu200_PHOBOSdNdeta.pkl',
            'AuAu200_PHOBOSv2eta.pkl',
            'AuAu200_pTvn.pkl',
            'AuAu19p6_dNdy.pkl',
            'AuAu19p6_PHOBOSdNdeta.pkl',
            'AuAu19p6_pTvn.pkl',
            'AuAu7.7_dNdy.pkl',
            'AuAu7.7_pTvn.pkl']

# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

event_data = []
for dataset in datasets_posterior:
    current_path = path_data + dataset
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)

    # Get the first event from the posterior dataset
    sorted_keys = sorted(data.keys())
    test_data = {event_id: data[event_id] for event_id in sorted_keys}
    event_data.append(test_data)

for dataset in event_data[1:]:
    for event in sorted_keys:
        # Get the 'obs' array for the current event
        obs_array_new = dataset[event]['obs']
        
        # Extend the 'obs' array of the first dataset with the values from the others
        event_data[0][event]['obs'] = np.concatenate((event_data[0][event]['obs'], obs_array_new), axis=1)

# Save separated data to pickle files
with open(f'{path_output}all_points_all_observables.pkl', 'wb') as pf1:
    pickle.dump(event_data[0], pf1)

# Delete parameters 16 and 17 from pkl files (bulk_max_rhob2,bulk_max_rhob4)

In [ ]:
path_data = '../data/'
path_output = '../data_new/'
datasets = ['AuAu7.7_dNdy.pkl',
            'AuAu7.7_pTvn.pkl',
            'AuAu19p6_dNdy.pkl',
            'AuAu19p6_PHOBOSdNdeta.pkl',
            'AuAu19p6_pTvn.pkl',
            'AuAu200_dNdy.pkl',
            'AuAu200_PHOBOSdNdeta.pkl',
            'AuAu200_PHOBOSv2eta.pkl',
            'AuAu200_pTvn.pkl',
            'AuAu7.7_logdNdy.pkl',
            'AuAu19p6_logdNdy.pkl',
            'AuAu19p6_logPHOBOSdNdeta.pkl',
            'AuAu200_logdNdy.pkl',
            'AuAu200_logPHOBOSdNdeta.pkl'
            ]

# Check if the output folder exists, if not, create it
if not os.path.exists(path_output):
    os.makedirs(path_output)

for dataset in datasets:
    current_path = path_data + dataset
    with open(current_path, "rb") as pf:
        data = pickle.load(pf)

    print(f"The dataset {dataset} has length {len(data)}.")
    # Separate data based on event ID
    sorted_event_ids = sorted(data.keys(), key=lambda x: int(x))
    first_data = {event_id: data[event_id] for event_id in sorted_event_ids}
    print("Parameters before =",len(first_data[f'{sorted_event_ids[0]}']['parameter']))

    for point in range(len(sorted_event_ids)):
        first_data[f'{sorted_event_ids[point]}']['parameter'] = np.delete(first_data[f'{sorted_event_ids[point]}']['parameter'], [16,17])

    print("Parameters after =",len(first_data[f'{sorted_event_ids[0]}']['parameter']))
    # Save new data to pickle files
    with open(f'{path_output}{dataset.split(".p")[0]}.pkl', 'wb') as pf1:
        pickle.dump(first_data, pf1)